<a href="https://colab.research.google.com/github/martharegina/machinelearning/blob/main/intermediate_machinelearning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

data = pd.read_csv("/content/melbourne_data.csv")
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [9]:
y = data['Price']
X = data.drop(['Price'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [11]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
X_train.drop(cols_with_missing, axis=1, inplace=True)
X_valid.drop(cols_with_missing, axis=1, inplace=True)

In [13]:
# Cari kolom yang low cardinality (< 10 unique value)
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype=='object']

# Cari kolom yang numerical
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10864 entries, 12167 to 2732
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           10864 non-null  object 
 1   Method         10864 non-null  object 
 2   Regionname     10864 non-null  object 
 3   Rooms          10864 non-null  int64  
 4   Distance       10864 non-null  float64
 5   Postcode       10864 non-null  float64
 6   Bedroom2       10864 non-null  float64
 7   Bathroom       10864 non-null  float64
 8   Landsize       10864 non-null  float64
 9   Lattitude      10864 non-null  float64
 10  Longtitude     10864 non-null  float64
 11  Propertycount  10864 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 1.1+ MB


In [23]:
object_cols = low_cardinality_cols
object_cols

['Type', 'Method', 'Regionname']

In [20]:
data['Type'].unique()

array(['h', 'u', 't'], dtype=object)

In [21]:
data['Method'].unique()

array(['S', 'SP', 'PI', 'VB', 'SA'], dtype=object)

In [22]:
data['Regionname'].unique()

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

In [17]:
def score_dataset(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=100, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)

In [19]:
# Approach 1 (Drop Categorical Variables)

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE dari Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE dari Approach 1 (Drop categorical variables):
175703.48185157913


In [24]:
# Approach 2 (Original Encoding)

from sklearn.preprocessing import OrdinalEncoder

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE dari Approach 2 (Original Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE dari Approach 2 (Original Encoding):
165936.40548390493


In [41]:
# Approach 3 (One-Hot Encoding)

from sklearn.preprocessing import OneHotEncoder

# handle unknown to avoid errors when validation data contains classes that aren't represented in the training data
# sparse to ensure that encoded columns are returned as a numpy array

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]),
                             columns=OH_encoder.get_feature_names_out(object_cols))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]),
                             columns=OH_encoder.get_feature_names_out(object_cols))

OH_cols_train.head()

,Type_h,Type_t,Type_u,Method_PI,Method_S,Method_SA,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
# Put back index
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE dari Approach 3 (One-Hot Encoding):")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE dari Approach 3 (One-Hot Encoding):
166089.4893009678


In [43]:
OH_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10864 entries, 12167 to 2732
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Rooms                                  10864 non-null  int64  
 1   Distance                               10864 non-null  float64
 2   Postcode                               10864 non-null  float64
 3   Bedroom2                               10864 non-null  float64
 4   Bathroom                               10864 non-null  float64
 5   Landsize                               10864 non-null  float64
 6   Lattitude                              10864 non-null  float64
 7   Longtitude                             10864 non-null  float64
 8   Propertycount                          10864 non-null  float64
 9   Type_h                                 10864 non-null  float64
 10  Type_t                                 10864 non-null  float64
 11  Type